# Code 2: Generating Clipped Images using Created Buffers

The code takes the screenshot of the specified area of interest and clips it using the previously generated buffers. These clipped images are then saved in both their original format and as RGB converted versions in JPEG files.
In summary, this code snippet utilizes geospatial data processing to extract smaller image segments focused on areas with destroyed buildings. These images are subsequently employed in training the model.

**Import necessary libraries**

In [ ]:
import os
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from shapely.geometry import box
from PIL import Image

**Set working directory and output path**

In [ ]:
# Paths to input raster and shapefile
raster_path = ""
shapefile_path = ""

# Output directory for saving clipped images
output_dir = ""

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)


**Clip raster to buffers**

In [ ]:
# Read the shapefile using geopandas
gdf = gpd.read_file(shapefile_path)

# Define the target CRS (EPSG:3857)
target_crs = 'EPSG:3857'

# Open the raster file and reproject it
with rasterio.open(raster_path) as src:
    transform, width, height = calculate_default_transform(
        src.crs, target_crs, src.width, src.height, *src.bounds
    )
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': target_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    reproj_raster_path = ""
    with rasterio.open(reproj_raster_path, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=target_crs,
                resampling=Resampling.nearest
            )

# Clip the shapefile's geometries to the extent of the raster
raster_extent = (transform[2], transform[5], transform[2] + transform[0] * width, transform[5] + transform[4] * height)
raster_bbox = box(*raster_extent)
clipped_gdf = gdf[gdf.geometry.intersects(raster_bbox)]

# Save the clipped shapefile
clipped_gdf.to_file(clipped_shapefile_path)

# Open the raster file
with rasterio.open(reproj_raster_path) as src:
    for index, row in clipped_gdf.iterrows():
        # Get the geometry of the shapefile row
        geom = row['geometry']

        # Clip the raster using the geometry
        out_img, out_transform = mask(src, [geom], crop=True)

        # Get metadata for the clipped image
        out_meta = src.meta.copy()

        # Update metadata for the clipped image
        out_meta.update({
            "height": out_img.shape[1],
            "width": out_img.shape[2],
            "transform": out_transform
        })

        # Save the clipped image as a JPEG
        output_path = os.path.join(output_dir, f"clip_{index}.jpg")
        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(out_img)

        # Optionally, convert the clipped image to RGB using PIL
        img = Image.open(output_path)
        img_rgb = img.convert("RGB")
        img_rgb.save(output_path)

        print(f"Saved clip {index} as {output_path}")
